In [1]:
import os
import tempfile
import urllib

import pandas as pd

from helpers.cell_type_naming import nice_to_weirds, weird_to_nice

In [2]:
truth = pd.read_csv(
    "gs://liulab/csx_example_files/groundtruth_Melanoma_Tirosh_et_al_SuppFig3b-d.txt",
    sep="\t",
    index_col=0,
).rename_axis(index="sample_id").rename(columns=weird_to_nice)

truth

,Malignant,Endothelial,CAF,T CD8,NK,Macrophage,T CD4,B
sample_id,,,,,,,,
53,0.1280,0.0880,0.0320,0.2000,0.0800,0.0960,0.3760,0.0000
58,0.0000,0.0000,0.0000,0.6508,0.0317,0.0159,0.2857,0.0159
59,0.8710,0.0000,0.1129,0.0000,0.0000,0.0161,0.0000,0.0000
60,0.0448,0.0000,0.0000,0.1940,0.0498,0.0199,0.2139,0.4776
65,0.0755,0.0000,0.0000,0.4717,0.0000,0.0189,0.3396,0.0943
67,0.0000,0.0000,0.0000,0.3059,0.0118,0.0000,0.4588,0.2235
71,0.6835,0.0000,0.0000,0.1899,0.0000,0.0253,0.1013,0.0000
72,0.0000,0.0000,0.0000,0.1634,0.0065,0.0000,0.6013,0.2288
74,0.0000,0.0000,0.0000,0.6496,0.0073,0.0365,0.2117,0.0949


In [3]:
experiments_root = "liulab/csx_experiments/varying_parameters"

!tree -h -L 1 {os.path.join("/mnt/buckets", experiments_root)}
!tree -h -L 1 {os.path.join("/mnt/buckets", experiments_root, "defaults,single_cell_true")}

/mnt/buckets/liulab/csx_experiments/varying_parameters
├── [   0]  defaults
├── [   0]  refsample_from_defaults
├── [   0]  refsample_from_defaults,single_cell_true
├── [   0]  refsample_from_defaults,with_bmode
├── [   0]  refsample_from_webjob4
├── [   0]  refsample_from_webjob4,with_bmode
├── [   0]  sigmat_from_defaults
├── [   0]  sigmat_from_defaults,with_bmode
├── [   0]  sigmat_from_webjob4
├── [   0]  sigmat_from_webjob4,with_bmode
└── [   0]  with_bmode

11 directories, 0 files
/mnt/buckets/liulab/csx_experiments/varying_parameters/defaults,single_cell_true [error opening dir]

0 directories, 0 files


In [4]:
experiments = os.listdir(os.path.join("/mnt/buckets", experiments_root))
experiments

['defaults',
 'refsample_from_defaults',
 'refsample_from_defaults,single_cell_true',
 'refsample_from_defaults,with_bmode',
 'refsample_from_webjob4',
 'refsample_from_webjob4,with_bmode',
 'sigmat_from_defaults',
 'sigmat_from_defaults,with_bmode',
 'sigmat_from_webjob4',
 'sigmat_from_webjob4,with_bmode',
 'with_bmode']

In [41]:
def load_csx_results(experiment_uri):
    try:
        uri = os.path.join(experiment_uri, "CIBERSORTx_Results.txt")
        df = pd.read_csv(uri, sep="\t", index_col=0)
    except:
        uri = os.path.join(experiment_uri, "CIBERSORTx_Adjusted.txt")
        df = pd.read_csv(uri, sep="\t", index_col=0)
    df = df.iloc[:, :-3]  # ignore last three columns, which are test statistics
    df.rename(
        columns=weird_to_nice,
        inplace=True)
    df.rename_axis(
        columns="cell_type",
        index="sample_id",
        inplace=True)
    return df.stack()

In [47]:
preds = dict()
# preds["derek"] = results_derek
# preds["me"] = results_docker
for experiment in experiments:
    try:
        experiment_uri = os.path.join("gs://", experiments_root, experiment)
        preds[experiment] = load_csx_results(experiment_uri)
    except FileNotFoundError:
        print("no results for", experiment, experiment_uri)

df_preds = pd.concat(
    preds,
    names=["experiment"],
    axis=0
).rename("csx_fraction")

# df_preds = df_preds.unstack("experiment")

df_preds

experiment  sample_id  cell_type  
defaults    53         Malignant      0.046178
                       Endothelial    0.027261
                       CAF            0.058420
                       T CD8          0.286861
                       NK             0.164019
                                        ...   
with_bmode  94         T CD8          0.003151
                       NK             0.017730
                       Macrophage     0.037556
                       T CD4          0.468256
                       B              0.291373
Name: csx_fraction, Length: 1672, dtype: float64

In [39]:
truth = pd.read_csv(
    "gs://liulab/csx_example_files/groundtruth_Melanoma_Tirosh_et_al_SuppFig3b-d.txt",
    sep="\t",
    index_col=0
)
truth.rename_axis(columns="cell_type", index="sample_id", inplace=True)
truth.rename(columns=weird_to_nice, inplace=True)
truth = truth.stack().to_frame(name="true_fraction")
truth

true_fraction
sample_id cell_type                 
53        Malignant           0.1280
          Endothelial         0.0880
          CAF                 0.0320
          T CD8               0.2000
          NK                  0.0800
...                              ...
94        T CD8               0.2025
          NK                  0.0041
          Macrophage          0.0083
          T CD4               0.3306
          B                   0.2686

[152 rows x 1 columns]

In [53]:
truth.join(df_preds.unstack("experiment"), how="left").corr().sort_values(by="true_fraction", ascending=False)

,true_fraction,defaults,refsample_from_defaults,"refsample_from_defaults,single_cell_true","refsample_from_defaults,with_bmode",refsample_from_webjob4,"refsample_from_webjob4,with_bmode",sigmat_from_defaults,"sigmat_from_defaults,with_bmode",sigmat_from_webjob4,"sigmat_from_webjob4,with_bmode",with_bmode
true_fraction,1.000000,0.940378,0.940378,0.940378,0.940901,0.941592,0.938370,0.940378,0.940901,0.941592,0.938370,0.940901
refsample_from_webjob4,0.941592,0.990750,0.990750,0.990750,0.991459,1.000000,0.995349,0.990750,0.991459,1.000000,0.995349,0.991459
sigmat_from_webjob4,0.941592,0.990750,0.990750,0.990750,0.991459,1.000000,0.995349,0.990750,0.991459,1.000000,0.995349,0.991459
"refsample_from_defaults,with_bmode",0.940901,0.995794,0.995794,0.995794,1.000000,0.991459,0.993899,0.995794,1.000000,0.991459,0.993899,1.000000
"sigmat_from_defaults,with_bmode",0.940901,0.995794,0.995794,0.995794,1.000000,0.991459,0.993899,0.995794,1.000000,0.991459,0.993899,1.000000
with_bmode,0.940901,0.995794,0.995794,0.995794,1.000000,0.991459,0.993899,0.995794,1.000000,0.991459,0.993899,1.000000
defaults,0.940378,1.000000,1.000000,1.000000,0.995794,0.990750,0.986880,1.000000,0.995794,0.990750,0.986880,0.995794
refsample_from_defaults,0.940378,1.000000,1.000000,1.000000,0.995794,0.990750,0.986880,1.000000,0.995794,0.990750,0.986880,0.995794
"refsample_from_defaults,single_cell_true",0.940378,1.000000,1.000000,1.000000,0.995794,0.990750,0.986880,1.000000,0.995794,0.990750,0.986880,0.995794
sigmat_from_defaults,0.940378,1.000000,1.000000,1.000000,0.995794,0.990750,0.986880,1.000000,0.995794,0.990750,0.986880,0.995794


In [75]:
validation_set = [x not in (80, 88) for x in df_preds.index.get_level_values("sample_id")]

In [83]:
truth\
    .join(
        df_preds.loc[:, validation_set, :].unstack("experiment")[["with_bmode"]],
        how="left")\
    .groupby(level="cell_type")\
    .corr()\
    .stack()\
    .loc[:, "true_fraction", "with_bmode"]\
    .sort_values(ascending=False)

cell_type
B              0.979903
Malignant      0.955684
Macrophage     0.948048
T CD4          0.937925
T CD8          0.890556
NK             0.883886
CAF            0.863670
Endothelial    0.861230
dtype: float64

In [ ]:
import plotly.io
plotly.io.renderers.default = "jupyterlab+png"

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
fig = px.bar(
    truth,
    x=truth.index.astype(str),
    y=truth.columns,
    title="True fractions of each mixture"
)

fig.show()